In [ ]:
!pip install pymorphy2

In [2]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Mounted at /content/drive


In [10]:
# открыть через колаб
# file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/Данные_ПиОА_2023_1семестр.xlsx"
file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/трек HARD.xlsx"
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [3]:
import AnalysisFunc
import BaseFuncs

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import datetime

In [ ]:
# открыть через виндовс
file_name = BaseFuncs.OpenFileNameWindows()
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [7]:
directory_name = "Учет контрольных"

In [11]:
# конвертирование данных от ИОТ
temp_row = ""
for sheet in xls_array:
    for name, group in sheet.groupby(["Название РМУП"]):
        if "Предмет контроля" not in group.columns:
            continue
        name_column_test = ""
        if "Контрольная работа" in set(list(group["Предмет контроля"])):
            name_column_test = "Контрольная работа"
        else:
            name_column_test = "Работа на учебной встрече"
        results = pd.DataFrame(columns=["ФИО", "Пол", "Команда", "Баллы", "Не сдал(-а)"])

        name_discipline = group.head(1)["Название РМУП"].values[0]
        try:
            if (group["Название встречи"].str.contains('Контрольная ').sum()==0 and name_column_test!="Контрольная работа"):
                continue
            print(name_discipline, end=":")
            print(name_column_test, set(list(group["Предмет контроля"])))

            max_scores_before_test = {}

            percent = 0
            count_presence = summ_presence = summ_goals = count_goals= 0
            startTime = datetime.now()
            # номер контрольной
            test_number = 1
            # # количество действий с кр (баллы и результаты до)
            for index, row in group.iterrows ():

                if (index%(len(group)//10)==0):
                    print(f"{percent}%. Записей в итоге: {len(results)}")
                    percent+=10

                if temp_row == "" or temp_row != row["ФИО студента"]:
                    test_number = 1
                    count_presence = summ_presence = summ_goals = count_goals= 0
                    temp_row = row["ФИО студента"]
                    results.loc[-1] = [temp_row, BaseFuncs.GetGender(temp_row), row["Команда"], row["Итог ТУ"], BaseFuncs.GetInfoLose(row, "Итог ТУ")] + [""]*(len(results.columns)-5)
                    results.index = results.index + 1  # shifting index

                if ("Контрольная " in row["Название встречи"] or "Контрольная работа" == row["Предмет контроля"]):
                    meeting_name = f"Контрольная работа {test_number}"

                    if (f"Контрольная работа {test_number}" not in list(results.columns)):
                        if (f"Посещение до {meeting_name}" not in list(results.columns)):
                            results[f"Посещение до {meeting_name}"] = 0.0
                            results[f"Баллы до {meeting_name}"] = 0.0
                            results[f"Количество баллов до {meeting_name}"] = 0.0
                            results[f"Процент баллов до {meeting_name}"] = 0.0
                        results[meeting_name] = 0.0
                    if (row["Предмет контроля"] == name_column_test):

                        results.loc[((results['ФИО'] == temp_row)), f"Контрольная работа {test_number}"] = row["Оценка за предметы контроля"] if row["Оценка за предметы контроля"]!="" else 0

                        if meeting_name not in max_scores_before_test:
                            max_scores_before_test[meeting_name] = summ_goals
                        elif max_scores_before_test[meeting_name] < summ_goals:
                            max_scores_before_test[meeting_name] = summ_goals

                        if count_presence == 0:
                            continue
                        results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                        results.loc[((results['ФИО'] == temp_row)), f"Баллы до {meeting_name}"] = summ_goals*1.0
                        results.loc[((results['ФИО'] == temp_row)), f"Количество баллов до {meeting_name}"] = count_goals
                        results.loc[((results['ФИО'] == temp_row)), f"Процент баллов до {meeting_name}"] = summ_goals*1.0
                        count_presence = summ_presence = summ_goals = count_goals= 0
                        test_number+=1


                elif (row["Предмет контроля"] != "Посещение"):
                    if not pd.isna(row["Оценка за предметы контроля"]):
                        goal = float(row["Оценка за предметы контроля"])
                        summ_goals += float(row["Оценка за предметы контроля"])
                        if goal>0.0:
                            count_goals += 1
                else:
                    count_presence += 1
                    if row["Оценка за предметы контроля"]=="П":
                        summ_presence += 1
            results = results.fillna(0)
            results = results.replace("", 0)
            results = results.drop(results.columns[-4:], axis= 1)
            results = results.astype({"Баллы": float})

            del max_scores_before_test[f"Контрольная работа {test_number}"]
            for meeting_name, max_score in max_scores_before_test.items():
                if max_score>0:
                    results = BaseFuncs.ConvertFloatByTestName(results, meeting_name)
                    results[f"Процент баллов до {meeting_name}"] = results[f"Процент баллов до {meeting_name}"].div(max_score*1.0).round(2)

            print(f"Время выполнения: {datetime.now()-startTime}")
            print("Отдельное сохранение...")

            BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = f"{name_discipline}", results = results)
            print("Сохранено")
        except Exception as e:
            print(e)



Алгебра:Работа на учебной встрече {'Посещение', 'Работа на учебной встрече'}
0%. Записей в итоге: 94
10%. Записей в итоге: 189
20%. Записей в итоге: 285
30%. Записей в итоге: 380
40%. Записей в итоге: 475
50%. Записей в итоге: 570
60%. Записей в итоге: 666
70%. Записей в итоге: 760
80%. Записей в итоге: 856
90%. Записей в итоге: 951
Время выполнения: 0:00:14.893971
Отдельное сохранение...
Сохранено
Программирование и основы алгоритмизации 1:Контрольная работа {'Контрольная работа', 'Посещение', 'Работа на учебной встрече'}
0%. Записей в итоге: 84
10%. Записей в итоге: 180
20%. Записей в итоге: 276
30%. Записей в итоге: 372
40%. Записей в итоге: 468
50%. Записей в итоге: 564
60%. Записей в итоге: 661
70%. Записей в итоге: 756
80%. Записей в итоге: 852
90%. Записей в итоге: 948
Время выполнения: 0:00:14.552647
Отдельное сохранение...
Сохранено
Математический анализ:Контрольная работа {'Контрольная работа', 'Посещение', 'Практическое задание', 'Письменный ответ', 'Работа на учебной встреч

In [ ]:
# очистка названий колонок и замена на первую строку в данны от Modeus
for sheet in xls_array:
    print(sheet.columns)
    sheet.columns = sheet.loc[0, :].values.flatten().tolist()
    sheet.drop(index=[0], inplace=True)
    sheet = sheet.reset_index(drop=True)

In [9]:
results = pd.DataFrame(columns=["Команды", "Направление", "Пол", "Баллы", "Не сдал(-а)"])
print(results.columns)

for sheet in xls_array:
    print(f"Записей: {len(sheet)}")
    startTime = datetime.now()
    percent = 0

    # try:
    max_scores_before_test = {}

    for index, row in sheet.iterrows ():
        if (index%(len(sheet)//10)==0):
            print(f"{percent}%. Записей в итоге: {len(results)}")
            percent+=10
        results.loc[-1] = [row["Учебные команды"], row["Направление подготовки"], BaseFuncs.GetGender(row["Обучающийся"]), row["Итог текущ."], BaseFuncs.GetInfoLose(row, "Итог текущ.")] + [""]*(len(results.columns)-5)
        results.index = results.index + 1

        count_presence = summ_presence = count_kontr = 0
        summ_goals = count_goals= 0
        for i in range(5, len(sheet.columns.to_list())):
            if sheet.columns.to_list()[i]!="Контрольная работа":
                if sheet.columns[i]=="Присутствие":
                    count_presence+=1
                    if (row[i]=="П"):
                        summ_presence+=1
                elif sheet.columns[i]=="Работа на учебной встрече":
                    if row[i]!="" and row[i]:
                        goal = (float(row[i]) if "." in str(row[i]) else float(str(row[i]).replace(",", ".")))
                        if not pd.isna(goal):
                            summ_goals += goal
                            count_goals += 1
            else:
                count_kontr+=1
                meeting_name = f"Контрольная работа {count_kontr}"

                if meeting_name not in max_scores_before_test:
                    max_scores_before_test[meeting_name] = summ_goals

                elif max_scores_before_test[meeting_name] < summ_goals:
                    max_scores_before_test[meeting_name] = summ_goals

                if (f"Посещение до {meeting_name}" not in list(results.columns)):
                    results[f"Посещение до {meeting_name}"] = 0
                    results[f"Баллы до {meeting_name}"] = 0
                    results[f"Количество баллов до {meeting_name}"] = 0.0
                    results[f"Процент баллов до {meeting_name}"] = 0
                    results[meeting_name] = 0

                results.loc[results.index[-1], f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                results.loc[results.index[-1], f"Баллы до {meeting_name}"] = summ_goals
                results.loc[results.index[-1], f"Количество баллов до {meeting_name}"] = count_goals
                results.loc[results.index[-1], f"Процент баллов до {meeting_name}"] = summ_goals
                results.loc[results.index[-1], meeting_name]  = float(row[i])

                count_presence = summ_presence = summ_goals = count_goals= 0

    results = results.fillna(0)
    results = results.astype({"Баллы": float})

    for meeting_name, max_score in max_scores_before_test.items():
        results[f"Процент баллов до {meeting_name}"] = results[f"Процент баллов до {meeting_name}"].div(max_score*1.0).round(2)
        results = BaseFuncs.ConvertFloatByTestName(results, meeting_name)

    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = "Программирование и основы алгоритмизации тест", results = results)
    print("Сохранено")
    # except Exception as e:
    #     print("Ошибка:", e)

Index(['Команды', 'Направление', 'Пол', 'Баллы', 'Не сдал(-а)'], dtype='object')
Записей: 961
0%. Записей в итоге: 95
10%. Записей в итоге: 191
20%. Записей в итоге: 287
30%. Записей в итоге: 383
40%. Записей в итоге: 479
50%. Записей в итоге: 575
60%. Записей в итоге: 671
70%. Записей в итоге: 767
80%. Записей в итоге: 863
90%. Записей в итоге: 959
Время выполнения: 0:00:04.532645
Отдельное сохранение...
Сохранено
